In [1]:
%matplotlib inline
import inspect, os, sys, copy, pytz, re, glob, random, praw, csv
import simplejson as json
import pandas as pd
from dateutil import parser
import datetime
import matplotlib.pyplot as plt   # Matplotlib for plotting
import matplotlib.dates as md
import numpy as np
import seaborn as sns
from collections import Counter, defaultdict
import re, urllib

from pathlib import Path
from datetime import datetime
import logging

utc=pytz.UTC

ENV = "production"
os.environ['CS_ENV'] = 'production'
BASE_DIR = "/home/nathan/CivilServant"
FILE_BASE_DIR="/home/nathan/reddit_archive/"
sys.path.append(BASE_DIR)
subreddit_id = "2qr7i"

LOG_PATH = str(Path(BASE_DIR, "logs", "praw_messenger_%s.log" % ENV))
logging.basicConfig(filename=LOG_PATH, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

# ARCHIVE_DIR = "/home/nathan/reddit_archive/03.2017"

# with open(os.path.join(BASE_DIR, "config") + "/{env}.json".format(env=ENV), "r") as config:
#   DBCONFIG = json.loads(config.read())

# ### LOAD SQLALCHEMY
# from sqlalchemy import create_engine
# from sqlalchemy.orm import sessionmaker
# from sqlalchemy import text, and_, or_
# from app.models import Base, SubredditPage, FrontPage, Subreddit, Post, ModAction, Experiment
# from utils.common import PageType

# db_engine = create_engine("mysql://{user}:{password}@{host}/{database}".format(
#     host = DBCONFIG['host'],
#     user = DBCONFIG['user'],
#     password = DBCONFIG['password'],
#     database = DBCONFIG['database']))
# DBSession = sessionmaker(bind=db_engine)
# db_session = DBSession()

### LOAD PRAW
import reddit.connection
conn = reddit.connection.Connect(base_dir=BASE_DIR)
r = conn.connect(use_db_keys=False)

#notebook_dir = os.getcwd()
#os.chdir(notebook_dir)
#import praw.errors


### FILTER OUT DEPRECATION WARNINGS ASSOCIATED WITH DECORATORS
# https://github.com/ipython/ipython/issues/9242
#import warnings
#warnings.filterwarnings('ignore', category=DeprecationWarning, message='.*use @default decorator instead.*')

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Logging to /home/nathan/CivilServant/logs/CivilServant_production.log


substituting https://oauth.reddit.com for https://api.reddit.com in url
GET: https://oauth.reddit.com/api/v1/me.json
POST: https://api.reddit.com/api/v1/access_token/
substituting https://oauth.reddit.com for https://api.reddit.com in url
GET: https://oauth.reddit.com/api/v1/me.json
substituting https://oauth.reddit.com for https://api.reddit.com in url
GET: https://oauth.reddit.com/api/v1/me.json


# Get list of study participants
We avoid archiving these accounts for privacy reasons

# Construct and personalize the message

## Construct the message template

In [2]:
MESSAGE_SUBJECT = "Welcome! Help r/feminism understand first-time commenters with 3 questions"
MESSAGE_TEMPLATE = """Dear {username},

Hello and welcome from r/feminism! 

Might you answer a 2 minute, 3-question survey to help us learn more about your experience? We're' currently working with researchers at Princeton University to improve newcomer experiences.

[Learn more and answer the survey here]({url}). Thanks! 

-- [CivilServant](http://civilservant.io/) and the [r/feminism moderators](https://www.reddit.com/r/feminism/about/moderators)
"""

In [3]:
print(MESSAGE_TEMPLATE.format(username="natematias", 
                              url="https://docs.google.com/forms/d/e/1FAIpQLSc1Atz_YmZVZIBILyVRsnZYHT3KrGqEseM-oUyz6dLllEfcVg/viewform?usp=pp_url&entry.360321409={username}".format(
                                  username="natematias")))

Dear natematias,

Hello and welcome from r/feminism! 

Might you answer a 2 minute, 3-question survey to help us learn more about your experience? We're' currently working with researchers at Princeton University to improve newcomer experiences.

[Learn more and answer the survey here](https://docs.google.com/forms/d/e/1FAIpQLSc1Atz_YmZVZIBILyVRsnZYHT3KrGqEseM-oUyz6dLllEfcVg/viewform?usp=pp_url&entry.360321409=natematias). Thanks! 

-- [CivilServant](http://civilservant.io/) and the [r/feminism moderators](https://www.reddit.com/r/feminism/about/moderators)



In [4]:
# r.send_message("natematias", MESSAGE_SUBJECT, 
#                MESSAGE_TEMPLATE.format(username="natematias",
#                                        url="https://docs.google.com/forms/d/e/1FAIpQLSc1Atz_YmZVZIBILyVRsnZYHT3KrGqEseM-oUyz6dLllEfcVg/viewform?usp=pp_url&entry.360321409={username}".format(
#                                             username="natematias")))

## Add the ability to validate users

In [5]:
def get_reddit_user(username):
    try:
        user_dict = r.get_redditor(username, fetch=True).json_dict
        logging.info("User %s found: %s" %(username, str(user_dict)))
        return user_dict
    except praw.errors.NotFound as e:
        logging.error("User not found: %s" % username)
    except Exception as e:
        logging.exception("Failed to retrieve user")

In [6]:
import time
send_log_dir = "/home/nathan/CivilServant-Analysis/SOC412/feminism"
send_log_filename = "followup_survey_experiment_12.csv"
def sent_message_previously(username):
    with open(os.path.join(send_log_dir, send_log_filename), "r") as f:
        send_log = []
        for row in csv.DictReader(f):
            send_log.append(row)
            if(username == row['username']):
                return True
    return False

## Add the ability to send messages

In [7]:
## RETURN TRUE IF MESSAGE WAS SENT SUCCESSFULLY
def send_reddit_message(**message_data):
    user = message_data["username"]    
    message = MESSAGE_TEMPLATE.format(**message_data)    
    try:
        logging.info("Sending a message to user %s with data %s" % (user, str(message_data)))
        response = r.send_message(user, MESSAGE_SUBJECT, message, raise_captcha_exception=True)
        if response["errors"]:
            logging.error("Error in response when sending a message to user %s: %s" % (user, str(response)))
            return False
        else:
            logging.info("Message successfully sent to user %s" % user)
            return True
    except praw.errors.InvalidCaptcha as e:
        logging.exception("Message sending requires a captcha")
        logging.error(e.response)
        return False
    except Exception as e:
        logging.exception("Failed to send message to %s" % user)
        return False

## Wrap the messenger function with anything survey-specific

In [8]:
def send_survey_message(**message_data):
    message = MESSAGE_TEMPLATE.format(**message_data)        
    sent_status = send_reddit_message(**message_data)
    with open(os.path.join(send_log_dir,send_log_filename), "a") as f:
        print("APPENDING TO FILE: {0}".format(message_data))
        f.write(",".join(map(str, [
            message_data['username'],
            True, #account exists
            message_data['get_info_date'],
            message_data['deleted'],
            message_data['url'],
            sent_status
        ])) + "\n")
        f.close()
    
    logging.info("url: %s" % message_data["url"])
    logging.info("get_info_data: %s" % message_data["get_info_date"])
    logging.info("deleted: %s" % message_data["deleted"])
    

In [9]:
def decide_and_send_to_user(username):
    if(sent_message_previously(username)):
        logging.info("User %s previously received a message" % username)
        return None
    else:
        time.sleep(2)
        user = get_reddit_user(username)

    if user:
        send_survey_message(
            username = username,
            get_info_date = str(datetime.utcnow()),
            deleted = False,
            url="https://docs.google.com/forms/d/e/1FAIpQLSc1Atz_YmZVZIBILyVRsnZYHT3KrGqEseM-oUyz6dLllEfcVg/viewform?usp=pp_url&entry.360321409={username}".format(
                username=username))
    else:
        with open(os.path.join(send_log_dir, send_log_filename), "a") as f:
            f.write(",".join(map(str, [
                username,
                False,
                str(datetime.utcnow()),
                None,
                None,
                False
            ])) + "\n")
            f.close()

# Load Survey Recipients

In [10]:
## LOAD RECIPIENTS FROM A CSV FILE
## THEN USE THE CODE BELOW TO SEND THEM THINGS
filename = "experiment_12_eligible_screen_names-08.02.2018.csv"
eligible_usernames = []
with open(os.path.join("..","inputs", filename), "r") as f:
    for row in csv.DictReader(f):
        eligible_usernames.append(row['screen_name'])
print("Found {0} eligible usernames".format(len(eligible_usernames)))

Found 802 eligible usernames


# Send Surveys to Participants

In [11]:
for username in eligible_usernames:
    decide_and_send_to_user(username)

User i-love-vaginas previously received a message
User beverwood previously received a message
User rlfrlf3094 previously received a message
User BobbieEyesUnclouded previously received a message
User awkqueered previously received a message
User DeathRayRobot previously received a message
User fierce_crab previously received a message
User 86MaisonJardin86 previously received a message
User screwmykarma previously received a message
User feministengineer previously received a message
User UserLogic previously received a message
User halfshoe previously received a message
User majeric previously received a message
User goonipers previously received a message
User 3_lov33-_id previously received a message
User joiveu previously received a message
User Yoshithemacabre previously received a message
User dwades00 previously received a message
User Dr_Trumps_Wild_Ride previously received a message
User BelowBingo previously received a message
User Zerotai previously received a message
User 

User kelizabethn previously received a message
User drkalmenius previously received a message
User GuyLoki previously received a message
User busy-belief previously received a message
User bluespudgrl previously received a message
User me33do previously received a message
User secretcharm previously received a message
User the_engineeress previously received a message
User haGaaaaaaaaaa-etc previously received a message
User whimsicalwheelbarrow previously received a message
User jenlew92 previously received a message
User eliaswestgaard previously received a message
User lilbb69 previously received a message
User NexArcX previously received a message
User DAMN_MY_AUNT_IS_HOT previously received a message
User blue_arc previously received a message
User karaoke_G previously received a message
User adamyones previously received a message
User Huckdog previously received a message
User catparlee previously received a message
User gangman731 previously received a message
User Aug11 previo

User Limb0State previously received a message
User buggaboo76 previously received a message
User heyitsstevie previously received a message
User theheshetheythatis previously received a message
User punkchopsticks previously received a message
User Traintoclimb512 previously received a message
User Bookislovakia previously received a message
User Restless_Red previously received a message
User eccentricaunt previously received a message
User Boibewbs25 previously received a message
User TijuanaSlushBucket previously received a message
User XtinaTeixeira previously received a message
User badmemes123 previously received a message
User nightowlnutter previously received a message
User zenthrowaway17 previously received a message
User outdistop previously received a message
User rundair previously received a message
User taigsc previously received a message
User ThatSpinosaurid previously received a message
User Dying_Bloodbirds previously received a message
User Vanessa79 previously rece

User wholesomefcknmrcls previously received a message
User idiocracyistooreal previously received a message
User ulubulu previously received a message
User HotMarketingGirl previously received a message
User TabbyCat1993 previously received a message
User _AileenW_ previously received a message
User TotesMessenger previously received a message
User NotArmchairAttorney previously received a message
User mr2jay previously received a message
User looksmaxxingcurry previously received a message
User nobredfruit previously received a message
User geenasq previously received a message
User shroedingerscook previously received a message
User movementlocation previously received a message
User Joie7994 previously received a message
User AngryFeministKnitter previously received a message
User hypnoticarmpit previously received a message
User Garrbear909 previously received a message
User Smileforcaroline previously received a message
User humblesmoker previously received a message
User egr3giou

User missetacarinae previously received a message
User Magnolia704 previously received a message
User Retired_Cheese previously received a message
User facebeneathanonymity previously received a message
User GrilledCheese248 previously received a message
User notakers400 previously received a message
User Mariofan231 previously received a message
User blandboringplain previously received a message
User anothermaninyourlife previously received a message
User a_rao1215 previously received a message
User Autobots_rollouts previously received a message
User SlyPhilosophersStone previously received a message
User PurpleHazedMuffins previously received a message
User PepeTheFootLettuce previously received a message
User Buzzlightyear1267 previously received a message
User luke_1602 previously received a message
User iamtheonetheone123 previously received a message
User LordOfTheREEES previously received a message
User cat_mom1 previously received a message
User GCnat previously received a me

...

Message successfully sent to user sandy_cheeki_breeki
APPENDING TO FILE: {'username': 'sandy_cheeki_breeki', 'url': 'https://docs.google.com/forms/d/e/1FAIpQLSc1Atz_YmZVZIBILyVRsnZYHT3KrGqEseM-oUyz6dLllEfcVg/viewform?usp=pp_url&entry.360321409=sandy_cheeki_breeki', 'deleted': False, 'get_info_date': '2019-03-23 13:48:43.778756'}
url: https://docs.google.com/forms/d/e/1FAIpQLSc1Atz_YmZVZIBILyVRsnZYHT3KrGqEseM-oUyz6dLllEfcVg/viewform?usp=pp_url&entry.360321409=sandy_cheeki_breeki
get_info_data: 2019-03-23 13:48:43.778756
deleted: False
User islcastaway1986 found: {'has_verified_email': True, 'is_employee': False, 'created': 1432941680.0, 'icon_img': 'https://www.redditstatic.com/avatars/avatar_default_07_0079D3.png', 'subreddit': None, 'hide_from_robots': False, 'verified': False, 'id': 'nsk5a', 'created_utc': 1432941680.0, 'link_karma': 25, 'name': 'islcastaway1986', 'pref_show_snoovatar': False, 'is_friend': False, 'is_mod': False, 'has_subscribed': True, 'is_gold': False, 'comment_kar

substituting https://oauth.reddit.com for https://api.reddit.com in url
GET: https://oauth.reddit.com/user/islcastaway1986/about/.json
substituting https://oauth.reddit.com for https://api.reddit.com in url
POST: https://oauth.reddit.com/api/compose/.json


Message successfully sent to user islcastaway1986
APPENDING TO FILE: {'username': 'islcastaway1986', 'url': 'https://docs.google.com/forms/d/e/1FAIpQLSc1Atz_YmZVZIBILyVRsnZYHT3KrGqEseM-oUyz6dLllEfcVg/viewform?usp=pp_url&entry.360321409=islcastaway1986', 'deleted': False, 'get_info_date': '2019-03-23 13:48:47.883363'}
url: https://docs.google.com/forms/d/e/1FAIpQLSc1Atz_YmZVZIBILyVRsnZYHT3KrGqEseM-oUyz6dLllEfcVg/viewform?usp=pp_url&entry.360321409=islcastaway1986
get_info_data: 2019-03-23 13:48:47.883363
deleted: False
User SeniorOctopod found: {'has_verified_email': True, 'is_employee': False, 'created': 1548264942.0, 'icon_img': 'https://www.redditstatic.com/avatars/avatar_default_01_94E044.png', 'subreddit': {'description': '', 'icon_size': [256, 256], 'public_description': '', 'link_flair_enabled': False, 'icon_img': 'https://www.redditstatic.com/avatars/avatar_default_01_94E044.png', 'subreddit_type': 'user', 'is_default_banner': True, 'banner_size': None, 'display_name': 'u_Senior

substituting https://oauth.reddit.com for https://api.reddit.com in url
GET: https://oauth.reddit.com/user/SeniorOctopod/about/.json
substituting https://oauth.reddit.com for https://api.reddit.com in url
POST: https://oauth.reddit.com/api/compose/.json


Message successfully sent to user SeniorOctopod
APPENDING TO FILE: {'username': 'SeniorOctopod', 'url': 'https://docs.google.com/forms/d/e/1FAIpQLSc1Atz_YmZVZIBILyVRsnZYHT3KrGqEseM-oUyz6dLllEfcVg/viewform?usp=pp_url&entry.360321409=SeniorOctopod', 'deleted': False, 'get_info_date': '2019-03-23 13:48:52.137569'}
url: https://docs.google.com/forms/d/e/1FAIpQLSc1Atz_YmZVZIBILyVRsnZYHT3KrGqEseM-oUyz6dLllEfcVg/viewform?usp=pp_url&entry.360321409=SeniorOctopod
get_info_data: 2019-03-23 13:48:52.137569
deleted: False
